# CS246 - Homework 1

## Question 1

### Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Get the data

In [ ]:
id='1cnwlg_eFEzNcKrI2UVNKiCVInoXfiAd2'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('soc-LiveJournal1Adj.txt')

Import Spark

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import itertools

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
data = sc.textFile('/content/soc-LiveJournal1Adj.txt')

In [ ]:
'''Hàm make_tuple nhận đầu vào là một chuỗi văn bản và chuyển đổi nó thành một friends_list. 
Mỗi cặp là một tuple gồm hai phần tử, đại diện cho hai friends, và một giá trị số nguyên, đại diện cho mối quan hệ giữa hai friends đó.'''
def make_tuple(line):
  tokens = line.split('\t')

  if len(tokens) == 1 or tokens[0] == '':
    return []

  user = int(tokens[0])
  friends = tokens[1].split(',')

  user_friend_pairs = []
  mutual_friend_pairs = []

  for friend in friends:
    if friend != '':
      user_friend_pairs.append(((user, int(friend)), 0))

  for i in range(0, len(friends) - 1):
    for j in range(i + 1, len(friends)):
      mutual_friend_pairs.append(((int(friends[i]), int(friends[j])), 1))
      mutual_friend_pairs.append(((int(friends[j]), int(friends[i])), 1))

  return user_friend_pairs + mutual_friend_pairs

In [ ]:
relationship_pairs = data.flatMap(lambda line: make_tuple(line))
already_friends = relationship_pairs.filter(lambda relationship: relationship[1] == 0)

mutual_friends_pairs = relationship_pairs.subtractByKey(already_friends).\
                       reduceByKey(lambda a, b: a + b).\
                       map(lambda x: (x[0][0], (x[0][1], x[1]))).\
                       groupByKey().\
                       mapValues(list)

In [ ]:
'''Hàm recommend_new_friends nhận đầu vào là một tuple gồm một số nguyên (đại diện cho ID) 
và một danh sách các ID lạ (đại diện cho các ID khác mà ID hiện tại chưa có mối liên quan với nó), 
và tùy chọn một số nguyên n để chỉ ra số lượng ID được đề xuất là bạn mới. Hàm trả về một tuple, 
với phần đầu tiên là số nguyên đại diện cho ID, và phần thứ hai là một danh sách gồm n ID được đề xuất là bạn mới.'''

def recommend_new_friends(user_and_strangers, n = 10):
  user, list_of_strangers = user_and_strangers

  ordered_strangers_list = sorted(list_of_strangers, key = lambda stranger_num_mutual_friends: (-stranger_num_mutual_friends[1], stranger_num_mutual_friends[0]))[:n]

  recommendation = []
  for k, v in ordered_strangers_list:
    recommendation.append(k)

  return user, recommendation

In [ ]:
#xử lý RDD
result = mutual_friends_pairs.map(lambda user_and_strangers: recommend_new_friends(user_and_strangers)).\
         map(lambda user_recommendations: "{}\t{}".format(user_recommendations[0], ",".join(map(lambda x: str(x), user_recommendations[1])))).\
         collect()

In [ ]:
user_IDs = ['924', '8941', '8942', '9019', '9020', '9021', '9022', '9990', '9992', '9993']

for user_id in user_IDs:
  for line in result:
    user, recommendations = line.split('\t')
    if user == user_id:
      print(line)

924	439,2409,6995,11860,15416,43748,45881
8941	8943,8944,8940
8942	8939,8940,8943,8944
9019	9022,317,9023
9020	9021,9016,9017,9022,317,9023
9021	9020,9016,9017,9022,317,9023
9022	9019,9020,9021,317,9016,9017,9023
9990	13134,13478,13877,34299,34485,34642,37941
9992	9987,9989,35667,9991
9993	9991,13134,13478,13877,34299,34485,34642,37941
